Using CardiffNLP's RoBERTA model for sensitive content (trained on twitter)
https://huggingface.co/cardiffnlp/twitter-roberta-large-sensitive-multilabel

Idea is to determine edge-cases of offensive content. I use a sliding window to compute the "offensiveness" of each segment, then check edge-case words for offensive context


In [ ]:
import pandas as pd
import os

In [139]:
# Import some df of lyrics
df = pd.read_csv('logs\\regulate-medium.en-all.csv')

lyrics = df['word'].tolist()
print(lyrics)

# These are the labels identified by 
labels = ['conflictual', 'profanity', 'drugs', 'sex', 'selfharm', 'spam', 'not-sensitive']

def zero_df(df, labels):    
    for label in labels:
        df[label] = 0.0

    return df

df = zero_df(df, labels)
df

['regulators', 'we', 'regulate', 'any', 'stealing', 'of', 'his', 'property', 'were', 'damn', 'good', 'too', 'but', 'you', 'cant', 'be', 'any', 'geek', 'off', 'the', 'street', 'gotta', 'be', 'handy', 'with', 'the', 'steal', 'if', 'you', 'know', 'what', 'i', 'mean', 'earn', 'your', 'keep', 'regulators', 'mauna', 'it', 'was', 'a', 'clear', 'black', 'night', 'a', 'clear', 'white', 'moon', 'warren', 'g', 'was', 'on', 'the', 'streets', 'trying', 'to', 'consume', 'some', 'skurts', 'for', 'the', 'e', 'so', 'i', 'could', 'get', 'some', 'phones', 'rolling', 'in', 'my', 'ride', 'chilling', 'all', 'alone', 'just', 'hit', 'the', 'east', 'side', 'of', 'the', 'lbc', 'on', 'a', 'mission', 'trying', 'to', 'find', 'mr', 'warren', 'g', 'seen', 'a', 'car', 'full', 'of', 'girls', 'aint', 'no', 'need', 'to', 'tweak', 'all', 'you', 'skirts', 'know', 'whats', 'up', 'with', '213', 'so', 'i', 'hooked', 'the', 'left', 'on', '21', 'and', 'louis', 'some', 'brother', 'shooting', 'down', 'so', 'i', 'said', 'lets', '

,word,start,end,prob,explicit,conflictual,profanity,drugs,sex,selfharm,spam,not-sensitive
0,regulators,4.12,4.60,0.766164,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,we,5.32,5.48,0.902294,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,regulate,5.48,5.80,0.991007,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,any,5.80,6.10,0.983366,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,stealing,6.10,6.48,0.878405,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,of,6.48,6.72,0.973185,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,his,6.72,6.92,0.976593,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,property,6.92,7.34,0.995175,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,were,7.52,7.74,0.971791,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,damn,7.74,8.04,0.963015,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch

# Define the model name
model_name = "cardiffnlp/twitter-roberta-large-sensitive-multilabel"

# AutoTokenizer loads the correct tokenizer for the model.
# AutoModelForSequenceClassification loads the model suitable for text classification.
print(f"Loading tokenizer for {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer loaded.")

print(f"Loading model for {model_name}...")
model = AutoModelForSequenceClassification.from_pretrained(model_name)
print("Model loaded.")

device = 'cuda' if torch.cuda.is_available() else 'cpu' 
print(f"Using device: {device}")
model.to(device) 

pipe = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=device, 
    top_k=None
    )

Loading tokenizer for cardiffnlp/twitter-roberta-large-sensitive-multilabel...
Tokenizer loaded.
Loading model for cardiffnlp/twitter-roberta-large-sensitive-multilabel...
Model loaded.
Using device: cuda


Device set to use cuda


In [140]:
n = len(df)
df = zero_df(df, labels)

for i in range(n):
    # Sliding window 
    left = max(i-2, 0)
    right = min(i+2, n-1)

    phrase = " ".join(df['word'][left:right])
    result = pipe(phrase)[0]
    
    for j in range(len(result)):
        #df[result[j]['label']][i] += result[j]['score']/10
        df.loc[i, result[j]['label']] += result[j]['score']
        

In [142]:
# Print the entire dataframe
pd.set_option('display.max_rows', None)
df

,word,start,end,prob,explicit,conflictual,profanity,drugs,sex,selfharm,spam,not-sensitive
0,regulators,4.12,4.60,0.766164,0,0.010057,0.006067,0.007174,0.004968,0.005449,0.007272,0.971260
1,we,5.32,5.48,0.902294,0,0.010119,0.007302,0.006591,0.004668,0.005451,0.007071,0.971623
2,regulate,5.48,5.80,0.991007,0,0.010388,0.006807,0.006536,0.004561,0.005421,0.008015,0.970654
3,any,5.80,6.10,0.983366,0,0.012044,0.005856,0.006070,0.004497,0.006560,0.006516,0.974145
4,stealing,6.10,6.48,0.878405,0,0.011213,0.004855,0.006891,0.004901,0.006506,0.007321,0.976644
5,of,6.48,6.72,0.973185,0,0.008477,0.006284,0.006751,0.006047,0.008157,0.005127,0.977691
6,his,6.72,6.92,0.976593,0,0.012215,0.004771,0.005781,0.005234,0.007032,0.006603,0.971238
7,property,6.92,7.34,0.995175,0,0.009120,0.005593,0.007314,0.005887,0.007019,0.006047,0.977880
8,were,7.52,7.74,0.971791,0,0.008479,0.016560,0.004992,0.006479,0.005181,0.004178,0.950894
9,damn,7.74,8.04,0.963015,0,0.007035,0.022689,0.006975,0.005937,0.004015,0.004033,0.940961


In [145]:
def flag_explicit(row):
    sometimes = ['smoke', 'gat', '44', 'forty', 'fourty', 'weed', 'ho', 'hoe', 'ass', 'bang', 'banged', 'jew', 'hos', 'hoes']
    if (row['word'] in sometimes) and (row['not-sensitive'] < .6):
        return 1

    else: 
        return row['explicit']

df['explicit'] = df.apply(flag_explicit, axis=1)

df_explicit = df[df['explicit'] == 1]
df_explicit

,word,start,end,prob,explicit,conflictual,profanity,drugs,sex,selfharm,spam,not-sensitive
351,gat,122.36,122.54,0.778134,1,0.006112,0.015154,0.023840,0.032294,0.032337,0.005415,0.350584
376,hoes,129.22,129.44,0.660014,1,0.023947,0.970146,0.015095,0.006993,0.006223,0.004738,0.030718
544,ass,227.24,227.54,0.982860,1,0.023587,0.949380,0.011014,0.006006,0.004214,0.003603,0.047187


In [146]:
df.to_csv("logs//regulate-lyrics.csv", index=False)